In [5]:
import torch
import torch.nn as nn
import torchvision
import torch.utils.data as Data
from CNN_torch import CNN

cnn = CNN();
print(cnn);

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (output): Linear(in_features=1568, out_features=10, bias=True)
)


In [6]:
# 加载数据集
train_data = torchvision.datasets.MNIST(
    root = "./minist", #训练数据保存路径
    train = True,      #True下载训练集，False下载测试集
    transform = torchvision.transforms.ToTensor(), #数据范围由（0-255）压缩到（0-1）
    download = False,  #是否需要从网上下载
);

test_data = torchvision.datasets.MNIST(
    root = "./minist",  #训练数据保存路径
    train = False,      #True下载训练集，False下载测试集
    download = False,   #是否需要从网上下载
);

test_x = torch.unsqueeze(test_data.data, dim=1).type(torch.FloatTensor)/255;
test_y = test_data.targets;


print(train_data.data.size());
print(test_data.data.size());

train_loader = Data.DataLoader(dataset=train_data,batch_size = 50,shuffle = True,num_workers=3);

torch.Size([60000, 28, 28])
torch.Size([10000, 28, 28])


In [3]:
# 网络训练
epoches = 1;
learning_rate = 0.001;
optimizer = torch.optim.Adam(cnn.parameters(),lr=learning_rate);
loss_function = nn.CrossEntropyLoss();

for epoch in range(epoches):
    print("Epoch:{}".format(epoch));
    for step,(batch_x,batch_y) in enumerate(train_loader):
        output = cnn(batch_x);
        loss = loss_function(output,batch_y);
        optimizer.zero_grad();
        loss.backward();
        optimizer.step();
        # 显示准确率
        if step%50 == 0:
            test_output = cnn(test_x);
            pred_y = torch.max(test_output,1)[1].data.numpy();
            accuracy = ( (pred_y==test_y.data.numpy()).astype(int).sum() )/float(test_y.size(0));
            print('Epoch:', epoch,'| train loss: %.4f' %loss.data.numpy(), '|test accuracy:%.2f'%accuracy);
            
        

Epoch:0
Epoch: 0 | train loss: 2.3017 |test accuracy:0.11
Epoch: 0 | train loss: 0.6129 |test accuracy:0.87
Epoch: 0 | train loss: 0.2488 |test accuracy:0.92
Epoch: 0 | train loss: 0.1313 |test accuracy:0.93
Epoch: 0 | train loss: 0.1224 |test accuracy:0.95
Epoch: 0 | train loss: 0.1483 |test accuracy:0.96
Epoch: 0 | train loss: 0.1218 |test accuracy:0.96
Epoch: 0 | train loss: 0.0983 |test accuracy:0.96
Epoch: 0 | train loss: 0.1637 |test accuracy:0.97
Epoch: 0 | train loss: 0.1071 |test accuracy:0.97
Epoch: 0 | train loss: 0.1665 |test accuracy:0.97
Epoch: 0 | train loss: 0.0577 |test accuracy:0.97
Epoch: 0 | train loss: 0.2115 |test accuracy:0.98
Epoch: 0 | train loss: 0.0297 |test accuracy:0.98
Epoch: 0 | train loss: 0.2445 |test accuracy:0.97
Epoch: 0 | train loss: 0.0925 |test accuracy:0.98
Epoch: 0 | train loss: 0.0717 |test accuracy:0.98
Epoch: 0 | train loss: 0.1019 |test accuracy:0.98
Epoch: 0 | train loss: 0.1297 |test accuracy:0.98
Epoch: 0 | train loss: 0.0497 |test accura

In [4]:
# 验证集结果预测
test_output = cnn(test_x[:10]);
pred_y = torch.max(test_output,1)[1].data.numpy().squeeze();
print(pred_y);
print(test_y[:10]);
print(test_output);

[7 2 1 0 4 1 4 9 5 9]
tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9])
tensor([[ -7.0069,  -4.6992,  -3.6207,   2.4665,  -9.3556,  -7.1956, -21.5144,
          14.1373,  -6.5343,   0.3045],
        [  0.0349,   2.2988,   8.8746,  -4.0080, -14.3413,  -7.6535,  -2.8469,
         -12.0265,  -0.7798, -17.1986],
        [ -5.4009,   7.4474,  -3.1242,  -4.0476,  -0.6666,  -5.8724,  -2.3300,
          -0.9481,  -2.4058,  -5.0741],
        [ 11.7698, -15.4288,  -4.1998,  -8.0303,  -5.9693,  -5.7000,   0.7348,
          -2.2754,  -5.7691,  -3.0541],
        [ -7.2858,  -6.5362,  -6.9520,  -7.5568,  10.4149,  -9.1863,  -4.8554,
           0.3696,  -5.1304,   0.4226],
        [ -6.2719,   9.1625,  -4.0100,  -4.8661,  -0.6066,  -8.4387,  -4.0926,
           0.1470,  -2.5393,  -5.2862],
        [-15.9141,  -2.8559,  -8.1673,  -6.7774,   7.0617,  -5.8389,  -9.1579,
          -2.6334,   0.1133,  -0.2386],
        [-10.6646,  -2.7301,  -3.2051,  -3.7911,  -0.0337,  -2.1204,  -9.3586,
          -3.1214,  -0.6865

In [5]:
from safetensors.torch import save_file
#保存模型为safetensors格式
save_file(cnn.state_dict(),"./cnn_minist.safetensors");

In [9]:
# 加载模型
from safetensors.torch import load_file
loaded_state_dict = load_file("cnn_minist.safetensors");
cnn.load_state_dict(loaded_state_dict);
loadModel_output = cnn(test_x[:10]);
print(loadModel_output);

tensor([[-0.0162, -0.0056, -0.0453, -0.0206, -0.1339, -0.0519, -0.1172, -0.1251,
         -0.0326, -0.0220],
        [-0.1079, -0.0184, -0.0224, -0.0761, -0.1324, -0.0333, -0.0767, -0.0937,
         -0.0043, -0.0959],
        [-0.0459,  0.0013, -0.0158, -0.0229, -0.1018, -0.0177, -0.0878, -0.0511,
         -0.0147, -0.0457],
        [-0.0754, -0.0587, -0.0762, -0.0191, -0.1425, -0.0555, -0.0859, -0.0983,
         -0.0452, -0.0286],
        [-0.0291, -0.0405, -0.0750, -0.0415, -0.1848, -0.0662, -0.1147, -0.1425,
         -0.0346, -0.0411],
        [-0.0552, -0.0107, -0.0268, -0.0285, -0.1387, -0.0103, -0.0922, -0.0630,
         -0.0164, -0.0404],
        [-0.0513, -0.0036, -0.0806, -0.0400, -0.1628, -0.0400, -0.0959, -0.1167,
         -0.0128, -0.0375],
        [-0.0689, -0.0227, -0.0539, -0.0192, -0.1479, -0.0511, -0.0918, -0.1147,
         -0.0201, -0.0028],
        [-0.0687, -0.0338, -0.1200, -0.0929, -0.2082, -0.0536, -0.1242, -0.1105,
         -0.0016, -0.0229],
        [-0.0655, -